In [1]:
import os
import tensorflow as tf
# Disable GPU usage
tf.config.set_visible_devices([], 'GPU')

# Check available devices
print("Available devices:", tf.config.list_physical_devices())


Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [2]:
!ulimit -Sv 16000000

import multiprocessing

num_cores = 2  # Set the number of CPUs you want to use
multiprocessing.set_start_method("fork", force=True)

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the GPT-2 tokenizer and model
model_name = "gpt2-med"  # You can use "gpt2-medium", "gpt2-large", or "gpt2-xl" for larger versions
# Load model directly

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2-medium")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
# Set the pad_token to eos_token (or add a custom pad_token if necessary)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Set EOS token as the pad_token

# Set the model to evaluation mode
model.eval()

# Define a prompt
prompt = "What is the capital of Denmark?"

# Encode the prompt (tokenize it) with padding
inputs = tokenizer(prompt, return_tensors="pt", padding=True)  # Ensure padding is applied if needed

# Add attention mask to the inputs
attention_mask = inputs['attention_mask']

# Generate text from the model with do_sample set to True for sampling-based generation
outputs = model.generate(
    inputs['input_ids'],
    attention_mask=attention_mask,  # Provide attention mask
    max_length=100,                  # Maximum number of tokens to generate
    num_return_sequences=1,         # Number of output sequences
    no_repeat_ngram_size=2,         # Prevent repetition
    top_p=0.92,                     # Nucleus sampling (controls randomness)
    top_k=50,                       # Top-k sampling
    temperature=0.85,               # Lower temperature makes text less random
    do_sample=True,                 # Enable sampling mode to use top_p, top_k, and temperature
    pad_token_id=tokenizer.pad_token_id  # Set pad token explicitly
)

# Decode the generated tokens back into text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)


What is the capital of Denmark?

The capital in Denmark is Vattenfall.
 "Vattenfalls" means "wealth" in Danish.


In [5]:
import psutil
import time

# Function to get memory usage
def get_memory_usage():
    memory = psutil.virtual_memory()
    return memory.percent

# Set the pad_token to eos_token (or add a custom pad_token if necessary)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Set EOS token as the pad_token

# Set the model to evaluation mode
model.eval()

# Define a prompt
prompt = "What is the capital of Denmark?"

# Encode the prompt (tokenize it) with padding
inputs = tokenizer(prompt, return_tensors="pt", padding=True)  # Ensure padding is applied if needed

# Add attention mask to the inputs
attention_mask = inputs['attention_mask']

# Track memory usage before running your model
before_memory = get_memory_usage()
print(f"Memory Usage Before Model: {before_memory}%")

# Generate text from the model with do_sample set to True for sampling-based generation
outputs = model.generate(
    inputs['input_ids'],
    attention_mask=attention_mask,  # Provide attention mask
    max_length=100,                  # Maximum number of tokens to generate
    num_return_sequences=1,         # Number of output sequences
    no_repeat_ngram_size=2,         # Prevent repetition
    top_p=0.92,                     # Nucleus sampling (controls randomness)
    top_k=50,                       # Top-k sampling
    temperature=0.85,               # Lower temperature makes text less random
    do_sample=True,                 # Enable sampling mode to use top_p, top_k, and temperature
    pad_token_id=tokenizer.pad_token_id  # Set pad token explicitly
)

# Decode the generated tokens back into text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

after_memory = get_memory_usage()
print(f"Memory Usage After Model: {after_memory}%")

print(generated_text)


Memory Usage Before Model: 24.6%
Memory Usage After Model: 25.4%
What is the capital of Denmark?

Capital of the Danish Republic is DKK 0.7 Billion (approximately USD 200 Million)
.
 From the list of countries:
And it is Denmark that has the lowest level of education in the world. In 2016, only 5.5% of people in Denmark had an education of 3-5 years. The highest share of those in education was in Russia, where only 20.4% had a tertiary education. But Denmark is


In [6]:
import time

# Set pad_token_id to eos_token_id
tokenizer.pad_token = tokenizer.eos_token

# Encode the prompt (tokenize it)
inputs = tokenizer("Mary had a little lamb", return_tensors="pt", padding=True, truncation=True)

# Add the attention mask to the inputs
attention_mask = inputs['attention_mask']

# Measure the time taken for text generation
start_time = time.time()
outputs = model.generate(
    inputs['input_ids'],
    attention_mask=attention_mask,  # Pass the attention mask
    max_length=50,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    top_p=0.92,
    top_k=50,
    temperature=0.85
)
end_time = time.time()

# Calculate latency
latency = end_time - start_time
print(f"Latency: {latency * 1000:.2f} ms")

# Decode the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.92` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Latency: 4112.46 ms
Mary had a little lamb, and she brought it to the Lord, saying, "I have brought you a lamb to eat with you." And the angel of the LORD said to her, "'I will bring you another lamb for you, but this


In [7]:
import time

batch_size = 8
prompt = "Mary had a little lamb"  # Ensure prompt is defined

# Set pad_token to eos_token as GPT-2 does not have a pad_token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the inputs with padding, truncation, and attention mask
inputs = tokenizer([prompt] * batch_size, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True)

# Check if the attention mask is included (it should be automatically if padding=True)
print(inputs.keys())  # This should include 'attention_mask' and 'input_ids'

start_time = time.time()
for _ in range(25):  # Run 100 iterations
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=50,
        pad_token_id=tokenizer.pad_token_id  # Explicitly set the pad_token_id
    )
end_time = time.time()

# Calculate throughput
throughput = (batch_size * 100) / (end_time - start_time)
print(f"Throughput: {throughput:.2f} samples/second")

dict_keys(['input_ids', 'attention_mask'])
Throughput: 2.46 samples/second


In [9]:
import time
!pip install datasets
from datasets import load_dataset

# Define batch size
batch_size = 8

# Load dataset (choose config appropriately)
gsm8k_dataset = load_dataset("gsm8k", name="main", split="test")
math_dataset = load_dataset("math_dataset", name="algebra__linear_1d", split="test")

# Choose the dataset to use, e.g., GSM8K
dataset = gsm8k_dataset  # Or replace with math_dataset

# Example preprocessing: Extract the 'question' field for prompts (adjust field name if needed)
prompts = dataset['question'][:batch_size]  # Get a batch of prompts

# Load tokenizer and model (GPT-2 example)
ttokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2-medium")

# Set padding_side to 'left' when initializing the tokenizer
tokenizer.padding_side = 'left'

# Set pad_token to eos_token (since GPT-2 doesn't have a pad_token)
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the inputs
inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True)

# Check if the attention mask is included
print(inputs.keys())  # This should include 'attention_mask' and 'input_ids'

# Run the model and measure latency
start_time = time.time()
for _ in range(10):  # Run 10 iterations (adjust as needed)
    outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_new_tokens=50)  # Only generate 50 new tokens
end_time = time.time()

# Calculate throughput
throughput = (batch_size * 10) / (end_time - start_time)
print(f"Throughput: {throughput:.2f} samples/second")

# Optionally: Print model outputs
for i, output in enumerate(outputs):
    print(f"Prompt: {prompts[i]}")
    print(f"Generated Output: {tokenizer.decode(output, skip_special_tokens=True)}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.0 MB/s eta 0:00:00


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

...

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


dict_keys(['input_ids', 'attention_mask'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Throughput: 0.37 samples/second
Prompt: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Generated Output: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?

"I make $2.50 a day," she says. "I don't make much money. I don't make much money. I don't make much money."

She says she's not sure how much she makes from
Prompt: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?
Generated Output: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in tot